In [ ]:
# wordcloud and boost specific words?
# statsmodels, ordinary least squares, pvalue



In [1]:
import pandas as pd
import numpy as np
from string import punctuation
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


In [2]:
fname = "../data/S1_8Sharktankpitchesdeals.csv"
data = pd.read_csv(fname)
# data["Number_Sharks"] = data["Kevin O'Leary"] + data["Lori Greiner"] + data["Mark Cuban"] + data["Robert Herjavec"] + data["Daymond John"] + data["Barbara Corcoran"] + data["Others"]
data

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Valuation,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,...,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,333333.0,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,...,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
1,101,College Foxes Packing Boxes,250000,0.25,1000000.0,Male,Lifestyle / Home,a packing and organizing service based on an a...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,101,Ionic Ear,1000000,0.15,6666667.0,Male,Software / Tech,an implantable Bluetooth device requiring surg...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,4600000.0,Male,Food and Beverage,a pie company,1,0,...,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
4,101,Wispots,1200000,0.10,12000000.0,Male,Business Services,an electronic hand-held device for waiting roo...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,824,Wallet Buckle,500000,0.10,5000000.0,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,...,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
700,826,Bridal Buddy,75000,0.10,750000.0,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,...,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN
701,826,Laid Brand,30000,0.30,100000.0,Male,Fashion / Beauty,hair-care products made with pheromones . Laid...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
702,826,Rocketbook,400000,0.10,4000000.0,Male,Software / Tech,a notebook that can scan contents to cloud ser...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [3]:
# Choose model output

# "Deal_Status"
# "Number_Sharks"
# "Deal_Shark 1"

# "Kevin O'Leary"
# "Barbara Corcoran"
# "Robert Herjavec"
# "Daymond John"
# "Lori Greiner"
# "Mark Cuban"
# "Others"
shark_to_model = "Deal_Shark 1"

In [4]:
# drop rows where shark_to_model is Nan
data=data.dropna(subset=[shark_to_model]).reset_index(drop=True)
data

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Valuation,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,...,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,333333.0,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,...,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
1,101,Mr. Tod's Pie Factory,460000,0.10,4600000.0,Male,Food and Beverage,a pie company,1,0,...,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
2,102,A Perfect Pear,500000,0.15,3333333.0,Female,Food and Beverage,a gourmet food business . a gourmet food business,1,0,...,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
3,102,Classroom Jams,250000,0.10,2500000.0,Male,Children / Education,a musical way to teach students Shakespeare . ...,1,1,...,0,1,1,1,0,Barbara Corcoran,Daymond John,Kevin O'Leary,Robert Herjavec,NaN
4,103,Stress Free Kids,250000,0.20,1250000.0,Female,Children / Education,a line of books that use stress relieving tech...,1,0,...,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,824,Rumi Spice,250000,0.05,5000000.0,Mixed,Food and Beverage,saffron imported from Afghanistan . Rumi Spice...,1,0,...,1,0,0,0,0,Mark Cuban,NaN,NaN,NaN,NaN
378,824,Thompson Tee,700000,0.07,10000000.0,Male,Fashion / Beauty,undershirts designed to block underarm sweat ....,1,0,...,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
379,824,Wallet Buckle,500000,0.10,5000000.0,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,...,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
380,826,Bridal Buddy,75000,0.10,750000.0,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,...,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN


In [5]:
# Custom tokenizer from class activity 22-1-4
stopwords = stopwords.words( 'english' ) + list(punctuation)
stemmer = PorterStemmer()
# Stemming
punc_list = list(punctuation)
def special_remove(word):
    if len(word)>2:
        return False
    for c in word:
        if c in punc_list:
            return True
    return False

# custom function that overrides default token generation
def custom_tokenizer(text):
    text = text.lower()
    words = nltk.word_tokenize(text)
    words = [stemmer.stem(w) for w in words if w not in stopwords+["..."]]
    # further remove words with a special char
    words = [w for w in words if not special_remove(w)]
    return words

activity_vectorizer = TfidfVectorizer(ngram_range=(1,2)
                            ,tokenizer=custom_tokenizer 
                            ,stop_words='english'
                            # ,min_df=2
                            ,strip_accents='unicode'
                            )

activity_vectors = activity_vectorizer.fit_transform(data["Pitched_Business_Desc"])
activity_df = pd.DataFrame(activity_vectors.toarray(), columns=activity_vectorizer.get_feature_names())
activity_df

C:\Users\atk285\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'thu', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ sun,“ sunglass,“ theyr,“ total,“ uv,“ wrap,”,” peopl,” potato,” prevent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
activity_vectorizer.get_feature_names()

['-not',
 '-not simpli',
 '10',
 '10 day',
 '10 net',
 '10,000',
 '10,000 run',
 '10-30',
 '10-30 minut',
 '100',
 '100 food',
 '100 guarante',
 '100 mile',
 '100 natur',
 '100 recycl',
 '100 time',
 '100 zinc',
 '12',
 '12 fashion',
 '123',
 '123 plush',
 '15',
 '15 flavor',
 '150',
 '150 90',
 '2',
 '2 ipad',
 '2-5',
 '2-5 age',
 '2-in-1',
 '2-in-1 spray',
 '2012',
 '2012 pipcorn',
 '20mph',
 '20mph 30mph',
 '21st',
 '21st centuri',
 '23',
 '23 ounc',
 '24',
 '24 mph',
 '2400',
 '2400 expert',
 '2400 sat',
 '26',
 '26 pound',
 '26g',
 '26g protein',
 '270',
 '270 shoe',
 '3',
 '3 design',
 '3 ipad',
 '3 pack',
 '3-d',
 '3-d design',
 '3-in-1',
 '3-in-1 nail',
 '30',
 '30 minut',
 '300,000',
 '300,000 fund',
 '30mph',
 '30mph travel',
 '35',
 '35 univers',
 '360',
 '360 degre',
 '360-degre',
 '360-degre pictur',
 '3d',
 '3d pop-up',
 '3d touch-bas',
 '4',
 '40+',
 '40+ mile',
 '5',
 '5 minut',
 '5-in-1',
 '5-in-1 babi',
 '5-in-1 convert',
 '50',
 '50 calib',
 '50 mph',
 '5g',
 '5g fru

In [7]:
# Choose features

# To include Pitch, comment in next line, and comment out the following line
# To exclude Pitch, comment out next line, and comment in the following line

selected_features = activity_df
# selected_features = pd.DataFrame([])

selected_features["Gender"]=data["Gender"]
selected_features["Category"]=data["Category"]
selected_features["Amount_Asked_For"]=data["Amount_Asked_For"]
selected_features["Exchange_For_Stake"]=data["Exchange_For_Stake"]
selected_features["Valuation"]=data["Valuation"]
selected_features

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ wrap,”,” peopl,” potato,” prevent,Gender,Category,Amount_Asked_For,Exchange_For_Stake,Valuation
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Female,Health / Wellness,50000,0.15,333333.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10,4600000.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Female,Food and Beverage,500000,0.15,3333333.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Children / Education,250000,0.10,2500000.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Female,Children / Education,250000,0.20,1250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Mixed,Food and Beverage,250000,0.05,5000000.0
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,700000,0.07,10000000.0
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,500000,0.10,5000000.0
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10,750000.0


In [8]:
selected_features = pd.get_dummies(selected_features)
selected_features

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,Category_Fitness / Sports / Outdoor,Category_Food and Beverage,Category_Green / Clean Tech,Category_Health / Wellness,Category_Lifestyle / Home,Category_Media / Entertainment,Category_Pet Products,Category_Software / Tech,Category_Travel,Category_Uncertain / Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
selected_features.columns

Index(['-not', '-not simpli', '10', '10 day', '10 net', '10,000', '10,000 run',
       '10-30', '10-30 minut', '100',
       ...
       'Category_Fitness / Sports / Outdoor', 'Category_Food and Beverage',
       'Category_Green / Clean Tech', 'Category_Health / Wellness',
       'Category_Lifestyle / Home', 'Category_Media / Entertainment',
       'Category_Pet Products', 'Category_Software / Tech', 'Category_Travel',
       'Category_Uncertain / Other'],
      dtype='object', length=8654)

In [10]:
model = MultinomialNB()
X_smote, y_smote = SMOTE().fit_sample(selected_features, data[[shark_to_model]].values.ravel())
X = X_smote
y = y_smote

# X = selected_features
# y = data[[shark_to_model]].values.ravel()
print(X.shape)
print(y.shape)

(553, 8654)
(553,)


In [11]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2, f_classif
# #apply SelectKBest class to extract top 10 best features
# bestfeatures = SelectKBest(score_func=f_classif, k=6)
# fit = bestfeatures.fit(X,y)
# dfscores = pd.DataFrame(fit.scores_)
# dfpvalues = pd.DataFrame(fit.pvalues_)
# dfcolumns = pd.DataFrame(X.columns)
# #concat two dataframes for better visualization 
# featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
# featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
# print(featureScores.nlargest(20,'Score'))  #print 10 best features

In [12]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2, f_classif
# #apply SelectKBest class to extract top 10 best features
# bestfeatures = SelectKBest(score_func=chi2, k=6)
# fit = bestfeatures.fit(X,y)
# dfscores = pd.DataFrame(fit.scores_)
# dfpvalues = pd.DataFrame(fit.pvalues_)
# dfcolumns = pd.DataFrame(X.columns)
# #concat two dataframes for better visualization 
# featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
# featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
# print(featureScores.nlargest(20,'Score'))  #print 10 best features

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [14]:
model.fit(X_train, y_train.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
y_pred = model.predict(X_test)

In [16]:
pd.DataFrame({"actual": y_test.reshape(-1), "prediction": y_pred.reshape(-1)})

,actual,prediction
0,Robert Herjavec,Daymond John
1,Daymond John,Barbara Corcoran
2,Robert Herjavec,Robert Herjavec
3,Kevin O'Leary,Lori Greiner
4,Mark Cuban,Kevin O'Leary
...,...,...
134,Kevin O'Leary,Kevin O'Leary
135,Barbara Corcoran,Barbara Corcoran
136,Daymond John,Lori Greiner
137,Daymond John,Barbara Corcoran


In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.18705035971223022

In [18]:
# Import ML models from sklearn
from sklearn.linear_model import LogisticRegression # Regression classifier
from sklearn.tree import DecisionTreeClassifier # Decision Tree classifier
from sklearn import svm # Support Vector Machine
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent Classifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # Random Forest and Gradient Boosting Classifier
from sklearn.naive_bayes import MultinomialNB # Naive Bayes Classifier 
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix # Some metrics to check the performance of the models

In [19]:
# Setting parameters for each algorithm - these are tunable to achieve max accuracy

Classifiers = {'LR':LogisticRegression(random_state=10,C=5,max_iter=300, solver='lbfgs')
               ,'DTC':DecisionTreeClassifier(random_state=10,min_samples_leaf=2)
               ,'RF':RandomForestClassifier(random_state=10,n_estimators=100,n_jobs=-1)
               # ,'GBC':GradientBoostingClassifier(random_state=10,n_estimators=400,learning_rate=0.2)
               ,'SGD':SGDClassifier(loss="hinge", penalty="l2")
               # ,'SVM':svm.SVC(kernel='linear', C=0.1)
               ,'NB':MultinomialNB(alpha=.05)
               }

In [20]:
Classifiers.keys()

dict_keys(['LR', 'DTC', 'RF', 'SGD', 'NB'])

In [21]:
# Create a pipeline so you can reuse the code

def ML_Pipeline(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(X, y.ravel())
    y_pred = clf.predict(X_test)
    # Accuracy = accuracy_score(y_test,y_pred)
    # Precision = metrics.precision_score(y_test,y_pred)
    # Recall = recall_score(y_test,y_pred)
    # Confusion_matrix = confusion_matrix(y_test,y_pred)
    # print('Classifier = '+(clf_name))
    # print('Accuracy = '+str(Accuracy))
    # print('Precision = '+str(Precision))
    # print('Recall = '+str(Recall))
    # print(Confusion_matrix)
    # print('==='*20) 
    return clf

In [24]:
clf_dict = []
for clf_name in Classifiers.keys():
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    Accuracy = accuracy_score(y_test,y_pred)
    # Precision = metrics.precision_score(y_test,y_pred)
    # Recall = recall_score(y_test,y_pred)
    Confusion_matrix = confusion_matrix(y_test,y_pred)
    clf_dict.append({'Classifier': clf_name, 'Accuracy': Accuracy 
    # ,'Precision': Precision, 'Recall': Recall
    })
    print('Classifier = '+(clf_name))
    print('Accuracy = '+str(Accuracy))
    # print('Precision = '+str(Precision))
    # print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20)


Classifier = LR
Accuracy = 0.1079136690647482
[[ 0  0  0  0  9  0 13]
 [ 0  0  0  0 10  0 17]
 [ 0  0  0  0  5  0 10]
 [ 0  0  0  0 12  0  9]
 [ 0  0  0  0  8  0  6]
 [ 0  0  0  3  5  0 12]
 [ 0  0  0  0 13  0  7]]
Classifier = DTC
Accuracy = 0.3669064748201439
[[ 6  3  3  6  4  0  0]
 [ 8  8  2  4  2  1  2]
 [ 2  2  7  0  4  0  0]
 [ 4  5  0  7  3  1  1]
 [ 2  4  2  2  1  0  3]
 [ 0  3  1  3  0 11  2]
 [ 1  3  0  0  4  1 11]]
Classifier = RF
Accuracy = 0.5683453237410072
[[15  2  1  1  3  0  0]
 [11 10  0  4  2  0  0]
 [ 4  0 10  0  1  0  0]
 [ 4  3  0 12  2  0  0]
 [ 6  1  1  3  3  0  0]
 [ 3  0  0  1  3 13  0]
 [ 3  0  1  0  0  0 16]]
Classifier = SGD
Accuracy = 0.14388489208633093
[[ 0  0  0  0  0 22  0]
 [ 0  0  0  0  0 27  0]
 [ 0  0  0  0  0 15  0]
 [ 0  0  0  0  0 21  0]
 [ 0  0  0  0  0 13  1]
 [ 0  0  0  0  0 20  0]
 [ 0  0  0  0  0 20  0]]
Classifier = NB
Accuracy = 0.18705035971223022
[[13  0  0  5  0  4  0]
 [15  2  1  5  0  4  0]
 [ 9  0  1  2  0  2  1]
 [ 7  1  0  5  0  

In [25]:
clf_df = pd.DataFrame(clf_dict)
clf_df

,Classifier,Accuracy
0,LR,0.107914
1,DTC,0.366906
2,RF,0.568345
3,SGD,0.143885
4,NB,0.187050


In [26]:
clf_df['Accuracy'].max()

0.5683453237410072

In [27]:
best_clf = clf_df.loc[clf_df['Accuracy'].idxmax(),'Classifier']
best_clf

'RF'

In [28]:
best_model = ML_Pipeline(best_clf)
best_model


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [29]:
from sklearn.externals import joblib
joblib.dump(best_model, f"{shark_to_model}_model.pkl")

C:\Users\atk285\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['Deal_Shark 1_model.pkl']

In [30]:
import pickle
pickle.dump(selected_features.columns,open(f"{shark_to_model}_vocab.pkl","wb"))

In [25]:
y_smote

array(['Barbara Corcoran', 'Barbara Corcoran', 'Robert Herjavec',
       'Barbara Corcoran', 'Barbara Corcoran', 'Others',
       'Barbara Corcoran', "Kevin O'Leary", 'Barbara Corcoran',
       "Kevin O'Leary", 'Others', 'Barbara Corcoran', 'Robert Herjavec',
       'Daymond John', 'Barbara Corcoran', 'Barbara Corcoran',
       'Barbara Corcoran', 'Barbara Corcoran', 'Daymond John',
       'Daymond John', "Kevin O'Leary", 'Others', 'Daymond John',
       'Barbara Corcoran', 'Barbara Corcoran', 'Others',
       'Barbara Corcoran', 'Barbara Corcoran', 'Daymond John', 'Others',
       'Daymond John', "Kevin O'Leary", 'Barbara Corcoran',
       'Daymond John', 'Barbara Corcoran', 'Mark Cuban',
       'Robert Herjavec', 'Mark Cuban', 'Barbara Corcoran',
       'Daymond John', 'Robert Herjavec', 'Barbara Corcoran',
       'Daymond John', 'Barbara Corcoran', 'Others', 'Daymond John',
       'Daymond John', 'Mark Cuban', 'Mark Cuban', 'Robert Herjavec',
       'Robert Herjavec', 'Lori Greiner'

In [45]:
# "Kevin O'Leary"
# "Barbara Corcoran"
# "Robert Herjavec"
# "Daymond John"
# "Lori Greiner"
# "Mark Cuban"
# "Others"
confusion_matrix(y_test,y_pred, labels=["Barabara Corcoran","Daymond John","Lori Greiner","Mark Cuban","Kevin O'Leary","Robert Herjavec","Others"])

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 4, 0, 4, 0, 1],
       [0, 0, 5, 0, 7, 1, 1],
       [0, 0, 1, 0, 8, 0, 0],
       [0, 0, 3, 0, 4, 1, 1],
       [0, 1, 2, 0, 3, 0, 2],
       [0, 0, 5, 0, 3, 0, 1]], dtype=int64)